In [1]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import animation, patches, cm, rc
from matplotlib import pyplot as plt
from Tomato3_dataStructure import Trial, Subject, Experiment
from Tomato3_helper import angle_overtake, lateral_overtake, overtake_rates, max_freewalk_spd, \
                            average_freewalk_spd, average_onset_spds, expansion_at, average_expansions, \
                            overtake_onset, time_to_pass, valid_trial, average_onset_delays
from helper import max_average, expansions, running_average
%matplotlib qt
rc('font', size=14)
# load data from binary file
with open('Tomato3_data.pickle', 'rb') as file:
    exp = pickle.load(file)


# Position, speed, accleration
## Experimental trials

In [3]:
'''animation'''
%matplotlib qt
subject = 10
trial = 38
exp.subjects[subject].trials[trial].play_trial(save=False, filtered=False)

In [4]:
'''plot trajectories, speeds, or accelerations of experimental trials'''
%matplotlib qt
frames = None
subject = [2]
trials = range(1,61)
filtered = True
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if i in subject and j in trials:
            if t.v0 == 1.2 and valid_trial(t):
#                 frames = list(range(t.f1, len(t.tstamps)))
                t.plot_trajectory(frames=frames, accelerations=False, links=True, filtered=filtered)
#                 t.plot_speeds(frames=frames, component='y', filtered=filtered)
#                 t.plot_accelerations(frames=frames, component='')
                label = 'overtake' if lateral_overtake(t, 0.2) else ''
                plt.text(0, 0, label) # overtake label

In [2]:
'''Plot all trajectories'''
%matplotlib qt
for i, s in exp.subjects.items():
    fig = plt.figure(figsize=(10, 8))
    fig.suptitle('subject ' + str(i))
    ax = []
    for i in range(6):
        ax.append(fig.add_subplot(2, 3, i+1))
        ax[i].set_xlim(-3, 3)
        ax[i].set_ylim(-1, 15)
        ax[i].set_title('v0 = ' + str((i + 8) / 10) + ' m/s')
    for j, t in s.trials.items():
        if valid_trial(t):
            lpos = t.get_positions('l')
            fpos = t.get_positions('f')
            i_ot = overtake_onset(t)
            i_ax = int(t.v0 * 10 - 8)
            overtake = lateral_overtake(t)
            color = 'r' if overtake else 'b'
            ax[i_ax].plot(lpos[t.f1:, 0], lpos[t.f1:, 1], c='k')
            ax[i_ax].plot(fpos[:, 0], fpos[:, 1], c=color)
            if overtake:
                ax[i_ax].scatter(fpos[i_ot, 0], fpos[i_ot, 1], c='r')

## Freewalk trials

In [2]:
'''Plot positions, speed, or acceleration of freewalk trials'''
%matplotlib qt
subject = 1
for i, t in exp.subjects[subject].freewalk.items():
    t.plot_speeds(distance=False)
#     t.plot_trajectory('f')
#     t.plot_accelerations()


In [5]:
'''average speed of freewalk trials using different window sizes'''
%matplotlib qt

for i, s in exp.subjects.items():
    fig = plt.figure(figsize=(12, 8))
    fig.suptitle('subject ' + str(i))
    ax = []
    for i in range(8):
        ax.append(fig.add_subplot(2, 4, i + 1))
#         ax[i].set_xlim(-3, 3)
        ax[i].set_ylim(1.0, 1.6)
    for j, t in s.freewalk.items():
        fspd = t.get_speeds('f')
        max_means = []
        for w in range(1, 10 * t.Hz + 1, t.Hz):
            max_means.append(max_average(fspd, w))
        ax[j-1].set_title('trial ' + str(j))
        ax[j-1].scatter(np.arange(1, 10 * t.Hz + 1, t.Hz) * 1.0 / t.Hz, max_means)
        

In [13]:
'''plot freewalk max_mean speed'''
%matplotlib qt
fig0 = plt.figure()
ax0 = fig0.add_axes(plt.axes())
ax0.set_xlim(0, 9)
ax0.set_ylim(0.8, 1.6)
ax0.plot([4.5, 4.5], [0.8, 1.6])

for i, s in exp.subjects.items():
    fig = plt.figure()
    ax = fig.add_axes(plt.axes())
    ax.set_xlim(0, 9)
    ax.set_ylim(0.8, 1.6)
    x = []
    y = []
    for j, t in s.freewalk.items():
        x.append(j)
        y.append(max_average(t.get_speeds('f'), 5 * t.Hz))
    ax.scatter(x, y)
    ax0.scatter(x, y)
    ax.plot([4.5, 4.5], [0.8, 1.6])
    

In [5]:
'''lateral deviation in freewalk trials'''
max_deviations = {}
for i, s in exp.subjects.items():
    max_deviation = 0
    for j, t in s.freewalk.items():
        _max = max(abs(t.get_positions('f')[:, 0]))
        if _max > max_deviation:
            max_deviation = _max
    max_deviations[i] = round(max_deviation, 3)
print(max_deviations)

{1: 0.102, 2: 0.097, 3: 0.207, 4: 0.205, 5: 0.281, 6: 0.144, 7: 0.121, 8: 0.148, 9: 0.394, 10: 0.304, 11: 0.128, 12: 0.12}


# Plot expansion and ttp

In [2]:
'''plot time to pass over time'''
%matplotlib qt
frames = None
subject = [2]
trials = range(1, 61)
filtered=True
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if i in subject and j in trials:
            if t.v0 in [1.1] and lateral_overtake(t, 0.2):
#                 frames = list(range(t.f1, len(t.tstamps)))
                fig = plt.figure()
                ax = plt.axes(xlim=(0, 12), ylim=(-1, 15))
                ttps = time_to_pass(t)
                ax.plot(t.tstamps_smooth, ttps)
                label = 'overtake' if lateral_overtake(t, 0.2) else ''
                ax.text(0, 0, label) # overtake label
                o = overtake_onset(t, tolerance=0.02)
                ax.scatter(t.get_time(filtered)[o], ttps[o], edgecolors='r', facecolors='none') # onset of overtaking

In [ ]:
'''plot expansion over time'''




# Overtaking frequency

In [4]:
'''Compare Overtake rate on pole and avatar'''

##########control panel###############
x_value = 'ratio' # 'leader', 'ratio', 'diff' or 'exp'
p_fspd_type = 'mean' # 'max', 'mean' or 'onset'
window = 2 # time window used for compute the mean speed
######################################
%matplotlib qt
# set the range of x axis
if x_value == 'ratio': xrange = (0.4, 1.2)
elif x_value == 'diff': xrange = (-0.2, 0.8)
elif x_value == 'exp': xrange = (0.1, 0.5)
elif x_value == 'leader': xrange = (0.7, 1.4)


# build figure for all in one plot
fig_all = plt.figure()
ax = plt.axes(xlim=xrange, ylim=(-0.1, 1.1))
ax.plot(xrange,[0.5, 0.5], 'k')
ax.set_ylabel('Probability of overtaking')
ax.set_xlabel('leader speed / fmean')
ax.set_title('All subjects')

for i, s in exp.subjects.items():
    
    # get speed scaler
    if p_fspd_type == 'max':
        p_fspd = max_freewalk_spd(s)
        p_fspd = [p_fspd] * 6
    elif p_fspd_type == 'mean':
        p_fspd = average_freewalk_spd(s, window)
        p_fspd = [p_fspd] * 6
    elif p_fspd_type == 'onset':
        p_fspd = average_onset_spds(s)
    
    # compute overtake rate
    rate = overtake_rates(s)
    
    # prepare x values and line color
    color = 'b' if s.leader == 'pole' else 'r'
    if x_value == 'ratio':
        x = [a / 10.0 / b for a, b in zip(range(8, 14), p_fspd)]
    elif x_value == 'diff':
        x = [b - a / 10.0 for a, b in zip(range(8, 14), p_fspd)]
    elif x_value == 'exp':
        x = average_expansions(s, relative=False)
    elif x_value == 'leader':
        x = [(a + 8) / 10 for a in range(6)]
        
    # plot overtake rate by scaled speed on all in one plot
    if s.leader == 'pole':
        pole = fig_all.axes[0].plot(x, list(rate.values()), c=color)
    elif s.leader == 'avatar':
        avatar = fig_all.axes[0].plot(x, list(rate.values()), c=color)
    
    # build figure for separate plots
    fig = plt.figure()
    ax = plt.axes(xlim=xrange, ylim=(-0.1, 1.1))
    ax.set_ylabel('Probability of overtaking')
    ax.set_xlabel(x_value + ' between ' + p_fspd_type + ' follower speed and leader speed')
    ax.set_title('subject ' + str(i))
    
    # plot overtake rate by scaled speed on separate plot
    ax.plot(x, list(rate.values()))
    ax.plot(xrange, [0.5, 0.5], 'k')
    
fig_all.axes[0].legend((pole[0],avatar[0]), ('pole', 'avatar'))

In [18]:
'''
Overtake rate by v0 by subject (12 plots). Overtaking is defined by angle 
and lateral deviation criteria.
'''
for i, s in exp.subjects.items():
    ot_angle = {0.8:0, 0.9:0, 1.0:0, 1.1:0, 1.2:0, 1.3:0}
    ot_lateral = {0.8:0, 0.9:0, 1.0:0, 1.1:0, 1.2:0, 1.3:0}

    for j, t in s.trials.items():
        if angle_overtake(t, 55):
            ot_angle[t.v0] += 0.1
        if lateral_overtake(t, 0.3):
            ot_lateral[t.v0] += 0.1
    fig = plt.figure()
    ax = plt.axes(xlim=(0.7, 1.4), ylim=(-0.1, 1.1))
    ax.plot(list(ot_angle.keys()),list(ot_angle.values()))
    ax.plot(list(ot_lateral.keys()),list(ot_lateral.values()))
    plt.title('subject ' + str(i))

# Onset of overtaking

In [3]:
'''onset of overtaking on lateral positions plot'''
%matplotlib qt
frames = None
subject = [5]
trials = [8]
filtered=True
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if i in subject and j in trials:
            if t.v0 >0 and valid_trial(t):
#                 frames = list(range(t.f1, len(t.tstamps)))
                t.plot_positions(frames=frames, component='x', filtered=filtered)
                label = 'overtake' if lateral_overtake(t, 0.2) else ''
                plt.text(0, 0, label) # overtake label
                plt.plot([0, 12], [0, 0], 'r') # zero line
                plt.plot([t.tstamps_smooth[t.f1], t.tstamps_smooth[t.f1]], [-2, 2], 'k') # leader appear
                o = overtake_onset(t, tolerance=0.02)
                fpos_x = t.get_positions('f')[:, 0]
                plt.scatter(t.get_time(filtered)[o], fpos_x[o], edgecolors='r', facecolors='none') # onset of overtaking

In [8]:
'''onset of overtaking on lateral speed plot'''
%matplotlib qt
frames = None
subject = [12]
trials = range(1,61)
filtered=True
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if i in subject and j in trials:
            if t.v0 == 1.3 and valid_trial(t):
#                 frames = list(range(t.f1, len(t.tstamps)))
                t.plot_speeds(frames=frames, component='x', filtered=filtered)
                label = 'overtake' if lateral_overtake(t, 0.2) else ''
                plt.text(0, 0, label) # overtake label
                fspd_x = t.get_velocities('f')[:, 0]
                plt.plot(t.tstamps_smooth, running_average(fspd_x), 'g', label='running average') # running average
                plt.plot([0, 12], [0, 0], 'r', label='zero') # zero line
                plt.plot([t.tstamps_smooth[t.f1], t.tstamps_smooth[t.f1]], [-1, 1], 'k') # leader appear
                plt.legend()
                o = overtake_onset(t, tolerance=0.02)
                fvel_x = t.get_velocities('f')[:, 0]
                plt.scatter(t.tstamps_smooth[o], fvel_x[o], edgecolors='r', facecolors='none') # onset of overtaking

In [15]:
'''Plot overtake onset'''
%matplotlib qt
onset = []
for i, s in exp.subjects.items():
    if i not in [5,  9]:
        for j, t in s.trials.items():
            if valid_trial(t) and lateral_overtake(t):
                _onset = (overtake_onset(t) - t.f1) / 90.0
                onset.append(_onset)

plt.scatter(np.random.normal(1, 0.04, size=len(onset)), onset)
print(onset[len(onset)//2])
print(sum(onset) / len(onset))
#         if abs(deviation) >= 0.2:
#             print('subject ', s.id, ' trial ', t.trial_id, ' deviation ', round(deviation, 2))


0.5444444444444444
1.4241532130421033


# Overtaking vs following

In [6]:
'''
Compare overtaking and following trials on fspd and fspd/max_fspd 
when leader appear.
'''
########Control Panel######
relative = False
difference = False
##########################


data = {0.8:[[],[]], 0.9:[[],[]], 1.0:[[],[]], 1.1:[[],[]], 1.2:[[],[]], 1.3:[[],[]]}

for i, s in exp.subjects.items():
    max_fspd = max_freewalk_spd(s)
    for j, t in s.trials.items():
        fspd = t.get_speeds('f')[t.f1]
        if difference:
            fspd -= t.v0
        if relative:
            fspd /= max_fspd
        if lateral_overtake(t):
            data[t.v0][1].append(fspd)
        else:
            data[t.v0][0].append(fspd)
            
for v0, fspd in data.items():
    # creat figure
    fig = plt.figure()
    if relative and difference:
        ax = plt.axes(ylim=(-0.5, 0.6))
    elif relative:
        ax = plt.axes(ylim=(0.3, 1.2))
    elif difference:
        ax = plt.axes(ylim=(-0.5, 0.9))
    else:
        ax = plt.axes(ylim=(0.5, 1.7))
    ax.set_title('leader speed = {}'.format(v0))
    ax.set_xticklabels(['follow', 'overtake'])
    ax.set_ylabel('follower speed when leader appears')
    
    # make boxplot
    ax.boxplot(fspd)
    
    # mark leader speed
    if not relative and not difference:
        ax.plot([0.5, 2.5], [v0, v0])

    # add scatter points on top of boxplot
    for i in [1,2]:
        x = np.random.normal(i, 0.04, size=len(fspd[i-1]))
        ax.plot(x, fspd[i-1], 'r.', alpha=0.2)


In [15]:
''' Compare overtaking and following trials on expansion '''
data = [[], []]
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if valid_trial(t):
            overtake = lateral_overtake(t)
            if overtake:
                idx = overtake_onset(t)
            else:
                idx = idx = t.f1 + int(1.44 * t.Hz) 
            e = expansion_at(t, relative=True, frame=idx)
            if overtake:
                data[1].append(e)
            else:
                data[0].append(e)

fig = plt.figure()
ax = plt.axes()
ax.set_xticklabels(['follow', 'overtake'])
ax.set_ylabel('relative rate of expansion at the onset of overtaking')

# make boxplot
ax.boxplot(data)

# add scatter points on top of boxplot
for i in [1,2]:
    x = np.random.normal(i, 0.04, size=len(data[i-1]))
    ax.plot(x, data[i-1], 'r.', alpha=0.2)

In [3]:
''' Compare overtaking and following trials on speed trajectories '''
subject = range(1,13)
duration = 2 # seconds after leader appear
for i, s in exp.subjects.items():
    if i in subject:
        fig = plt.figure(figsize=(10, 8))
        fig.suptitle('subject ' + str(i))
        ax = []
        for j in range(6):
            ax.append(fig.add_subplot(2, 3, j+1))
            ax[j].set_xlim(-1, duration + 1)
            ax[j].set_ylim(0.5, 2)
            ax[j].set_title('v0 = ' + str((j + 8) / 10) + ' m/s')
        for j, t in s.trials.items():
            if valid_trial(t):
                overtake = lateral_overtake(t)
                fspd_y = t.get_velocities('f')[:, 1]
    #             fspd = t.get_speeds('f')
                color = 'r' if overtake else 'b'
                ax[int(t.v0 * 10 - 8)].plot([a / t.Hz for a in range(duration * t.Hz)], fspd_y[t.f1:t.f1 + duration * t.Hz], c=color)
                if overtake:
                    ax[int(t.v0 * 10 - 8)].scatter((overtake_onset(t) - t.f1) / t.Hz, fspd_y[overtake_onset(t)], marker='o', c='r')

# Write to CSV file

In [6]:
'''write data to file'''
data = []
for i, s in exp.subjects.items():
    if i not in [5, 9]:
        fmax = max_freewalk_spd(s)
        fmean = average_freewalk_spd(s, 2)
        delays = average_onset_delays(s)
        print(delays)
        for j, t in s.trials.items():
            if valid_trial(t):
                overtake = lateral_overtake(t)
                lspd = t.v0
                if overtake:
                    idx = overtake_onset(t)
                else:
                    idx = t.f1 + delays[lspd]
                    idx2 = t.f1 + int(1.43 * t.Hz)
                fspd = t.get_speeds('f')
                fspd_s = fspd[idx]
                fspd0 = fspd[t.f1]
                fspd_c = fspd[idx2]
                dist_vec = t.get_positions('l')[idx] - t.get_positions('f')[idx]
                dist_y = dist_vec[1]
                dist = np.linalg.norm(dist_vec)
                ttp = time_to_pass(t)[idx]
                e = expansion_at(t, relative=False, frame=idx)
                e2 = expansion_at(t, relative=True, frame=idx)
                data.append([i, j,
                             lspd, fspd0, fmax, fmean, fspd_c, fspd_s,
                             dist, dist_y, ttp, e, e2,
                             int(overtake)])

df = pd.DataFrame(data, columns=['subj', 'trial',
                                   'lspd', 'fspd0', 'fmax', 'fmean', 'fspd_c', 'fspd_s', 
                                   'dist', 'dist_y', 'ttp', 'e', 'e2',
                                   'overtaking'])
df.to_csv(path_or_buf='overtaking.csv', index=False)



{0.8: 128, 0.9: 203, 1.0: 261, 1.1: 253, 1.2: 253, 1.3: 253}
{0.8: 58, 0.9: 57, 1.0: 80, 1.1: 167, 1.2: 192, 1.3: 192}
{0.8: 120, 0.9: 140, 1.0: 196, 1.1: 299, 1.2: 299, 1.3: 299}
{0.8: 177, 0.9: 250, 1.0: 250, 1.1: 250, 1.2: 250, 1.3: 250}
{0.8: 77, 0.9: 81, 1.0: 96, 1.1: 31, 1.2: 31, 1.3: 31}
{0.8: 90, 0.9: 124, 1.0: 81, 1.1: 205, 1.2: 205, 1.3: 205}
{0.8: 88, 0.9: 91, 1.0: 115, 1.1: 118, 1.2: 233, 1.3: 233}
{0.8: 102, 0.9: 133, 1.0: 105, 1.1: 87, 1.2: 133, 1.3: 133}
{0.8: 83, 0.9: 93, 1.0: 165, 1.1: 220, 1.2: 124, 1.3: 124}
{0.8: 66, 0.9: 65, 1.0: 82, 1.1: 69, 1.2: 184, 1.3: 158}


In [13]:
m1 = Logit(df['overtaking'], df[['lspd', 'fspd0', 'fmax']]).fit()
m1.summary()


Optimization terminated successfully.
         Current function value: 0.260667
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             overtaking   No. Observations:                  592
Model:                          Logit   Df Residuals:                      589
Method:                           MLE   Df Model:                            2
Date:                Mon, 02 Dec 2019   Pseudo R-squ.:                  0.6143
Time:                        14:49:53   Log-Likelihood:                -154.32
converged:                       True   LL-Null:                       -400.06
Covariance Type:            nonrobust   LLR p-value:                1.873e-107
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
lspd         -19.9022      1.676    -11.874      0.000     -23.187     -16.617
fspd0         11.3642      1.735      6.551      0.000       7.964      14.764
fmax           4.9977      1.408      3.551      0.000       2.239       7.756
==============================================================================
"""